This notebook relies on the following cmd line tools, ensure they are installed and in the system
- tippecanoe
- mapshaper
- aws cli

In [1]:
from pathlib import Path
import sys

scripts_dir = Path('../..').joinpath('src')
if scripts_dir not in sys.path:
    sys.path.insert(0, scripts_dir.resolve().as_posix())

In [2]:
from utils import downloadFile, unzipFile, Settings
# from mapshaper import Mapshaper # for the future
from mapbox_uploader import uploadToMapbox
from tippcanoe import mbtileGeneration
from pydantic import BaseModel
from functools import lru_cache
import logging
from IPython.lib import backgroundjobs as bg

In [3]:
logging.basicConfig(level=logging.INFO)
jobs = bg.BackgroundJobManager()

In [4]:
@lru_cache()
def get_settings():
    return Settings(data_directory=Path('/home/mamabauser/data'))

class VTPipeConfig(BaseModel):
    data_download_url: str
    name: str
    file_name: str
    folder_name: str
    settings: Settings = get_settings()


    def pipe(self):
        '''
        Pipeline to download, unzip, convert to mbtiles and upload to Mapbox
        '''

        folder_path = self.settings.data_directory.joinpath(self.folder_name)
        file_path = folder_path.joinpath(self.file_name)
        mbtiles_path = folder_path.joinpath(self.file_name).with_suffix(".mbtiles")
        
        folder_path.mkdir(parents=True, exist_ok=True)

        downloadFile(self.data_download_url, folder_path)
        unzipFile(file_path, folder_path)
        mbtileGeneration(folder_path.joinpath(self.file_name), mbtiles_path)
        #uploadToMapbox(mbtiles_path, self.name, self.settings.MAPBOX_USER, self.settings.MAPBOX_TOKEN)

    def execute(self):
        """Run the pipeline
        """
        try:
            self.settings.validate_config()
            return self.pipe()
        except ValueError as e:
            raise e


In [5]:
eez_PipeConfig = VTPipeConfig(
    data_download_url="https://www.marineregions.org/download_file.php?file=EEZ_land_v3.zip",
    name="eez",
    file_name="EEZ_land_v3.zip",
    folder_name="EEZ_land_v3",
)

In [6]:
eez_PipeConfig.execute()

INFO:utils:Downloading file.in /home/mamabauser/data/EEZ_land_v3/EEZ_land_v3.zip...
INFO:utils:Download Finish.


BadZipFile: File is not a zip file